In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import cm, ticker 

In [3]:
import numpy as np

In [4]:
import yt
import yt.units as u
import yt.utilities.physical_constants as const
from yt.fields.api import ValidateSpatial

/mnt/bwpy/single/usr/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
import sys
sys.path.insert(0, '/u/sciteam/liao/first_star/python/yt_related/yt_gamer_grackle')

In [6]:
from yt_gamer_grackle import *

In [7]:
units_override = {"length_unit": (1.0, "AU"),
                  "time_unit"  : (159.16319433669406, "yr"),
                  "mass_unit"  : (1.0e-6, "msun")}

In [8]:
## sobolev_9: 57
frame_n = format(58, '06d')

In [9]:
ds = yt.load('/u/sciteam/liao/scratch/fs256_gk_sobolev_10/Data_'+frame_n, units_override=units_override)

yt : [WARNING  ] 2019-08-21 12:18:12,322 Overriding code units: Use this option only if you know that the dataset doesn't define the units correctly or at all.
yt : [INFO     ] 2019-08-21 12:18:12,324 Overriding length_unit: 1 AU.
yt : [INFO     ] 2019-08-21 12:18:12,327 Overriding time_unit: 159.163 yr.
yt : [INFO     ] 2019-08-21 12:18:12,329 Overriding mass_unit: 1e-06 msun.
yt : [INFO     ] 2019-08-21 12:18:12,400 Parameters: current_time              = 0.805
yt : [INFO     ] 2019-08-21 12:18:12,402 Parameters: domain_dimensions         = [ 256 1024  512]
yt : [INFO     ] 2019-08-21 12:18:12,405 Parameters: domain_left_edge          = [  0.5   0.  -50. ]
yt : [INFO     ] 2019-08-21 12:18:12,408 Parameters: domain_right_edge         = [50.          6.28318531 50.        ]
yt : [INFO     ] 2019-08-21 12:18:12,411 Parameters: cosmological_simulation   = 0


In [10]:
N1, N2, N3 = ds.domain_dimensions

In [11]:
## manipulate ds
ds.periodicity = (True, True, True)
ds.mu = 2

In [12]:
add_popIII_field(ds, optical_depth=2)
add_cyl_field(ds)

/mnt/a/u/sciteam/liao/github/yt/yt/units/yt_array.py:1395: RuntimeWarning: invalid value encountered in power
  out=out, **kwargs)
/mnt/a/u/sciteam/liao/github/yt/yt/units/yt_array.py:1374: RuntimeWarning: overflow encountered in exp
  out_arr = func(np.asarray(inp), out=out, **kwargs)
/mnt/a/u/sciteam/liao/github/yt/yt/units/yt_array.py:1395: RuntimeWarning: invalid value encountered in true_divide
  out=out, **kwargs)
/mnt/a/u/sciteam/liao/github/yt/yt/units/yt_array.py:1395: RuntimeWarning: divide by zero encountered in true_divide
  out=out, **kwargs)
/mnt/a/u/sciteam/liao/github/yt/yt/units/yt_array.py:1395: RuntimeWarning: invalid value encountered in multiply
  out=out, **kwargs)
/mnt/a/u/sciteam/liao/github/yt/yt/units/yt_array.py:1374: RuntimeWarning: invalid value encountered in sqrt
  out_arr = func(np.asarray(inp), out=out, **kwargs)


In [13]:
## make sure theta direction is connected
ds.periodicity = (False, True, False)

In [14]:
frame_n_out = format(int(frame_n) - 57, '02d')  ## sobolev
frame_n_out

'01'

### Run clump finder
cf. https://yt-project.org/doc/analyzing/analysis_modules/clump_finding.html

In [15]:
from yt.data_objects.level_sets.api import *

In [16]:
ad = ds.all_data()

#### Initiate clump finder 

In [17]:
clump_fields = [("gas", "density"), ("gas", "epicycle_freq_square")]
master_clump = Clump(ad, clump_fields)

#master_clump = Clump(ad, ("gas", "density"))

TypeError: unhashable type: 'list'

#### add validator

In [ ]:
def _minimum_gas_mass(clump, min_mass):
    return (clump["gas", "cell_mass"].sum() >= min_mass)
add_validator("minimum_gas_mass", _minimum_gas_mass)

In [ ]:
master_clump.add_validator("min_cells", 8)
#master_clump.add_validator("gravitationally_bound", use_particles=False) #need velocity_x, velocity_y, etc.
master_clump.add_validator("minimum_gas_mass", ds.quan(1e-3, "Msun"))

#### run clump finder

In [ ]:
rho_min = 5e-12*(u.g/u.cm**3)
rho_max = ad["gas", "density"].max()
step    = 2.0

find_clumps(master_clump, rho_min, rho_max, step)

In [ ]:
leaf_clumps = master_clump.leaves

In [ ]:
leaf_clumps = master_clump.leaves

In [ ]:
print(leaf_clumps[0]["gas", "r"].in_units("au"))

In [ ]:
print(leaf_clumps[0]["gas", "velocity_z"])

#### Plot